# **Asteroid Prediction**


# Import Python Libraries

In [340]:
#from google.colab import drive
#drive.mount('/content/drive')

In [341]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Reading the Income Classification Dataset

In [342]:
#%cd /content/drive/My Drive/Data

In [343]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [344]:
df = pd.read_csv('/content/drive/MyDrive/Asteroid.csv',  on_bad_lines='skip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,14,15,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [345]:
df.head()

,full_name,a,e,G,i,om,w,q,ad,per_y,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# Column Renaming

In [346]:
readable_df = df.copy()
readable_df = readable_df.rename(columns={"a": "semi-major axis",
                            "e": "eccentricity",
                            "G": "Magnitude slope parameter",
                            "i": "Inclination with ecliptic plane",
                            "om": "Longitude of the ascending node",
                            "w": "argument of perihelion",
                            "q": "perihelion distance(au)",
                            "ad": "aphelion distance(au)",
                            "per_y": "Orbital period",
                            "data_arc": "data arc-span(d)",
                            "condition_code": "Orbit condition code",
                            "n_obs_used": "number of observations used",
                            "H": "Absolute Magnitude parameter",
                            "condition_code": "Orbit condition code",
                            })

In [347]:
readable_df.head()

,full_name,semi-major axis,eccentricity,Magnitude slope parameter,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# **Exploration (EDA) & Visualization**

# Handling missing values

In [348]:
readable_df.shape

(839736, 27)

**Drop nan rows (diameter col)**

In [349]:
readable_df = readable_df.dropna(axis=0, subset=['diameter'])

**Split a dataset into training and testing sets**

In [350]:
dtf_train, dtf_test = train_test_split(readable_df, test_size=0.2)

In [351]:
dtf_train.shape

(110144, 27)

In [352]:
dtf_test.shape

(27537, 27)

**missing values**

In [353]:
dtf_train.isna().sum()

full_name                               0
semi-major axis                         0
eccentricity                            0
Magnitude slope parameter          110047
Inclination with ecliptic plane         0
Longitude of the ascending node         0
argument of perihelion                  0
perihelion distance(au)                 0
aphelion distance(au)                   0
Orbital period                          0
data arc-span(d)                      113
Orbit condition code                    0
number of observations used             0
Absolute Magnitude parameter          614
diameter                                0
extent                             110134
albedo                                991
rot_per                            101179
GM                                 110133
BV                                 109356
UB                                 109385
IR                                 110143
spec_B                             109051
spec_T                            

In [354]:
dtf_train.dtypes

full_name                           object
semi-major axis                    float64
eccentricity                       float64
Magnitude slope parameter          float64
Inclination with ecliptic plane    float64
Longitude of the ascending node    float64
argument of perihelion             float64
perihelion distance(au)            float64
aphelion distance(au)              float64
Orbital period                     float64
data arc-span(d)                   float64
Orbit condition code                object
number of observations used          int64
Absolute Magnitude parameter       float64
diameter                            object
extent                              object
albedo                             float64
rot_per                            float64
GM                                 float64
BV                                 float64
UB                                 float64
IR                                 float64
spec_B                              object
spec_T     

In [355]:
dtf_train = dtf_train.astype({'diameter': 'float64'})
dtf_test= dtf_test.astype({'diameter': 'float64'})

In [356]:
categorical_variables = ['Orbit condition code', 'extent', 'spec_B', 'spec_T', 'neo', 'pha']

**contiguous cols with nan**

In [357]:
nan_cols_con = ['semi-major axis', 'aphelion distance(au)', 'Magnitude slope parameter', 'data arc-span(d)', 'Absolute Magnitude parameter', 
                'albedo', 'rot_per', 'GM', 'BV', 'UB', 'IR', 'moid', 'Orbital period']

# fill the NaN values with the mean




In [358]:
def fillnawithMean(nan_cols, dtf_train, dtf_test):
  for arg in nan_cols_con:
    dtf_train[arg] = dtf_train[arg].fillna(dtf_train[arg].mean())
    dtf_test[arg] = dtf_test[arg].fillna(dtf_train[arg].mean())


In [359]:
fillnawithMean(nan_cols_con,dtf_train, dtf_test)

In [360]:
dtf_train.isna().sum()

full_name                               0
semi-major axis                         0
eccentricity                            0
Magnitude slope parameter               0
Inclination with ecliptic plane         0
Longitude of the ascending node         0
argument of perihelion                  0
perihelion distance(au)                 0
aphelion distance(au)                   0
Orbital period                          0
data arc-span(d)                        0
Orbit condition code                    0
number of observations used             0
Absolute Magnitude parameter            0
diameter                                0
extent                             110134
albedo                                  0
rot_per                                 0
GM                                      0
BV                                      0
UB                                      0
IR                                      0
spec_B                             109051
spec_T                            

In [361]:
# dtf_train['pha'] = dtf_train['pha'].fillna(dtf_train['pha'].value_counts().index[0])
# dtf_test['pha'] = dtf_test['pha'].fillna(dtf_train['pha'].value_counts().index[0])

In [362]:
for arg in ['extent', 'spec_B', 'spec_T']:
  dtf_train[arg] = dtf_train[arg].fillna('U')
  dtf_test[arg] = dtf_test[arg].fillna('U')

In [363]:
dtf_train.isna().sum().sum()

0

In [364]:
dtf_test.isna().sum().sum()

0

**Drop 'full_name' col**

In [365]:
dtf_train = dtf_train.drop(['full_name'], axis=1)
dtf_test = dtf_test.drop(['full_name'], axis=1)

## Dealing with Categorical variables


In [366]:
categorical_data = dtf_train.select_dtypes(include=['object']).copy()
categorical_data_test = dtf_test.select_dtypes(include=['object']).copy()
categorical_data.head() 

,Orbit condition code,extent,spec_B,spec_T,neo,pha
19564,0,U,U,U,N,N
622659,9,U,U,U,N,N
37334,0,U,U,U,N,N
196928,0,U,U,U,N,N
350260,0,U,U,U,N,N


In [367]:
#dtf_train['Orbit condition code'].value_counts()

In [368]:
#dtf_test['Orbit condition code'].value_counts()

**"One Hot" encoding**

In [369]:
new_columns = pd.get_dummies(categorical_data, columns=categorical_data.columns, drop_first=True)
new_columns_test = pd.get_dummies(categorical_data_test, columns=categorical_data_test.columns, drop_first=True)
new_columns.head()

,Orbit condition code_1,Orbit condition code_2.0,Orbit condition code_3.0,Orbit condition code_4.0,Orbit condition code_5.0,Orbit condition code_6.0,Orbit condition code_7.0,Orbit condition code_8.0,Orbit condition code_9.0,Orbit condition code_0,...,spec_T_XDC,spec_T_XF,spec_T_XFCU,spec_T_XFU,spec_T_XSC,spec_T_XSCU,spec_T_XU,spec_T_Z,neo_Y,pha_Y
19564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
622659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Define the same columns on train and test**

In [370]:
new_columns, new_columns_test = new_columns.align(new_columns_test, join='left', axis=1)

In [371]:
new_columns = new_columns.fillna(0)
new_columns_test = new_columns_test.fillna(0)

**Drop original categorical cols**

In [372]:
dtf_train = dtf_train.drop(categorical_data.columns, axis = 1)
dtf_test = dtf_test.drop(categorical_data_test.columns, axis = 1)

In [373]:
model_df_train = pd.concat([dtf_train, new_columns], axis=1)
model_df_test = pd.concat([dtf_test, new_columns_test], axis=1)
model_df_train.head()

,semi-major axis,eccentricity,Magnitude slope parameter,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,data arc-span(d),...,spec_T_XD,spec_T_XD:,spec_T_XDC,spec_T_XF,spec_T_XFCU,spec_T_XFU,spec_T_XSC,spec_T_XSCU,spec_T_XU,spec_T_Z
19564,3.075658,0.300625,0.178763,23.409470,247.213951,54.975507,2.151039,4.000278,5.394055,17009.0,...,0,0,0,0,0,0,0,0,0,0
622659,2.497102,0.316965,0.178763,5.301327,124.435319,10.914294,1.705608,3.288596,3.946051,1.0,...,0,0,0,0,0,0,0,0,0,0
37334,2.391160,0.113889,0.178763,7.656416,100.416875,250.866668,2.118832,2.663487,3.697609,9062.0,...,0,0,0,0,0,0,0,0,0,0
196928,3.154738,0.205380,0.178763,22.489226,3.619891,134.378934,2.506816,3.802659,5.603419,5821.0,...,0,0,0,0,0,0,0,0,0,0
350260,3.039289,0.032858,0.178763,16.180644,358.558717,131.946047,2.939423,3.139154,5.298662,5399.0,...,0,0,0,0,0,0,0,0,0,0


In [374]:
model_df_train.shape

(110144, 190)

In [375]:
model_df_test.shape

(27537, 190)

# Asteroids

## Linear Regression

In [376]:
y_train = model_df_train['diameter']
X_train = model_df_train.drop(['diameter'], axis = 1)
y_test = model_df_test['diameter']
X_test = model_df_test.drop(['diameter'], axis = 1)

In [377]:
X_test.shape

(27537, 189)

In [378]:
X_train.isna().sum().sum()

0

In [379]:
multi_reg = LinearRegression()
multi_reg.fit(X_train , y_train)
pred = multi_reg.predict(X_test)

In [380]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.7356367255105545
Test Accuracy :  0.6511357181793286


## Random Forest Regression

In [338]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [339]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.7572537576163535
Test Accuracy :  0.4963696112780095
